In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from shutil import copyfile
import html2text
import re
import logging
import csv
import enchant
import glob
from bs4 import BeautifulSoup
from lxml import etree
from lxml import html
import gc

In [5]:
labor_discrim_folder = os.path.join(os.path.expanduser('~'), 'Dropbox', 'Indigenous', 'Saved RA Documents')

In [4]:
out_folder = os.path.join(os.getcwd(), 'html_files6' )

In [25]:
#Moving HTML Files to local directory
filenames = []
overwritten = []
for index, filename in enumerate(glob.iglob('/home/jason/Dropbox/**/*.h*', recursive=True)):
    basename = os.path.basename(filename)
    out_filename = "{}_{}".format(basename, index)
    file_dst = os.path.join(out_folder, out_filename)
    copyfile(filename, file_dst)

In [9]:
# unwritten = []
# clids = {}
# clid_names = []
parser = etree.HTMLParser()
new_out = os.path.join(os.getcwd(), 'html_files_clid' )
for index, file in enumerate(os.listdir(out_folder)):
    html_doc = os.path.join(out_folder, file)
    try:
        tree = etree.parse(html_doc, parser)
        clid = tree.xpath('/html/body/section/section/section/div[2]/p[1]//text()')[0].split(": ",1)[1]
        #clids[clid] = ''
        #clid_names.append(clid)
        out_name = "{}.html".format(clid)
        out_loc = os.path.join(new_out, out_name)
        copyfile(html_doc, out_loc)
    except Exception as e:
        #clids[file] = e
        pass
        
        
        

In [11]:
#Obtaining list of jbos we both possess
joblist_csv = os.path.join(os.getcwd(), 'Relevant_Files', 'Regular_Use', 'JobList.csv')
joblist = pd.read_csv(joblist_csv)
all_jobs = list(map(lambda s: s.replace("\"",""),joblist['clid'].values))
all_jobs_set = set(all_jobs)
my_jobs = [os.path.splitext(x)[0] for x in os.listdir(new_out)]
both_possess = all_jobs_set.intersection(my_jobs)


In [34]:
#Moving into a Data folder
out_path = os.path.join(os.getcwd(), 'Data')
for index, file in enumerate(both_possess_paths):
    out_loc = os.path.join(out_path, os.path.basename(file))
    copyfile(file, out_loc)

In [78]:
#Finding the mutually missing HTML files
my_jobs_set = set(my_jobs)
brig_miss= np.asarray(list(my_jobs_set.difference(all_jobs_set)))
me_miss = np.asarray(list(all_jobs_set.difference(my_jobs_set)))
with open('missing_html.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['brig_missing'])
    for i in brig_miss:
        writer.writerow([i])
with open('me_miss.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['me_missing'])
    for i in me_miss:
        writer.writerow([i])

In [84]:
log_path = os.path.join(os.getcwd(), 'cant_read1.log')
logging.basicConfig(filename = log_path, filemode = 'w')
logging.info("HTML File Could Not be read")
logger = logging.getLogger("HTMLOpenError")

corpus_nocheck = []
corpus_check = []
notaword = []
d = enchant.Dict("en_US")
# h = html2text.HTML2Text()
# h.ignore_links = True


In [85]:
#new_out = os.path.join(os.getcwd(), 'html_files5' )
data_dir = os.path.join(os.getcwd(), 'Data')
parser = etree.HTMLParser()
corpus_body = {}
for index, file in enumerate(os.listdir(data_dir)):
    html_doc = os.path.join(data_dir, file)
    try:
        tree = etree.parse(html_doc, parser)
        body_text = tree.xpath('//*[@id="postingbody"]//text()')
        body_text_str = ' '.join(body_text)
        body_text_clean = body_text_str.rstrip().replace("\n","")
        clid = tree.xpath('/html/body/section/section/section/div[2]/p[1]//text()')[0].split(": ",1)[1]
        corpus_body[clid] = body_text_clean
#         soup = BeautifulSoup(open(html_doc), "html.parser")
#         #print(soup.find(id="postingbody").text)
#         finding_text = soup.find(id="postingbody")
#         text = (finding_text.text).rstrip()
#         text_clean = re.sub(r'\W+', '', text)
#         corpus_body.append(text)
#         tree = html.fromstring(soup)
        
    except:
        print(index)
        logger.exception(html_doc)

In [86]:
logging.shutdown()
#reload(logging)

In [88]:
#Obtain our bag of words
vectorizer = CountVectorizer(stop_words = 'english')
X = vectorizer.fit_transform(corpus_body.values())
words = vectorizer.get_feature_names()

In [94]:
with open('words.csv', 'w') as f:
    writer = csv.writer(f)
    for i in words:
        writer.writerow([i])

In [114]:
# #Remove all non-words and present as dataframe
# words_only = np.asarray(words[2493:])
# words_only_values = X.toarray()[:,2493:]
# bag_of_words_df = pd.DataFrame(words_only_values, columns = words_only, index = corpus_body.keys())
# word_counts = bag_of_words_df.sum(axis=0)

In [118]:
# #Loading all dropped words
# dropped_words = pd.read_csv('dropped_words.csv')
# dropped_words_values = dropped_words['dropped_word'].dropna().values
# bag_of_words_df.drop(dropped_words_values, axis = 1, inplace = True) #Drop the dropped words
# bag_of_words_df.to_csv('bag_of_words_>10.csv') #Save occurrences greater than 10

In [135]:
#subsetting words only with 100+ occurrences
word_counts_df = pd.read_csv('word_counts.csv')
word_100 = word_counts_df[word_counts_df['count'] >= 100]['word'].values
bag_of_words_100 = bag_of_words_df[word_100]
bag_of_words_100.to_csv('bag_of_words_100.csv')

In [ ]:
# #Obtaining a subset of words with more than 100 occurrences
# bag_of_words_df
# with open('word_occurrences_100+.csv', 'r') as f:
#     csvreader = csv.reader(f)
#     next(csvreader, None)
#     word_choice = [row[0] for row in csvreader]
#     f.close()
# word_choice
# bag_of_words_df_subset = bag_of_words_df[word_choice]
# bag_of_words_df_subset.to_csv('bag_of_words_100+.csv')

## Acquiring Labels

In [ ]:
#Used to label as discriminated if callback for nonminority is 1 and callback for minority is zero
def discriminate(row):
    if row['callback_non_minority'] == 1.0 and row['callback_minority'] == 0 :
        val = 1
    else:
        val = 0
    return val

In [ ]:
#Preparing data
labor_discrim_data = os.path.join(os.getcwd(),'full_data - Jason.dta')

l_discrim_df = pd.read_stata(labor_discrim_data, index_col = 'clid', columns = ['clid', 'callback', 'indigenous'])
l_discrim_df.index = l_discrim_df.index.map(str)
l_discrim_nonminority = l_discrim_df[l_discrim_df['indigenous'] == 0].rename(index=str, 
                                                                          columns = {"callback": "callback_non_minority"}).drop(['indigenous'], axis = 1)
l_discrim_minority = l_discrim_df[l_discrim_df['indigenous'] == 1].rename(index=str, 
                                                                          columns = {"callback": "callback_minority"}).drop(['indigenous'], axis = 1)
l_discrim_fin = l_discrim_nonminority.join(l_discrim_minority, how='outer')
l_discrim_fin['discriminated'] = l_discrim_fin.apply(discriminate, axis = 1)
l_discrim_fin.to_csv('flattened_discriminated.csv')

In [ ]:
discrim_labels = pd.read_csv('flattened_discriminated.csv', index_col = 'clid')
bag_of_words_100 = pd.read_csv('bag_of_words_100+.csv', index_col = 'clid')

In [ ]:
possessed_ads = set(bag_of_words_100.index.values)
all_ads = set(discrim_labels.index.values)
labels_to_drop = (all_ads.difference(possessed_ads))
word_rows_to_drop = possessed_ads.difference(all_ads)
discrim_labels_possessed = discrim_labels.drop(labels_to_drop)
bag_of_words_possessed = bag_of_words_100.drop(word_rows_to_drop)

In [ ]:
bag_of_words_labels = bag_of_words_possessed.join(discrim_labels_possessed, how='outer').dropna()
bag_of_words_labels.to_csv('labor_discrim_whole.csv')

In [ ]:
#Getting inconsistency in amount between labeled ADS and the ads I possess
labeled_clid = pd.read_csv('flattened_discriminated.csv', index_col = 'clid')
clids = set([str(x).split('.')[0] for x in labeled_clid.index.values])
data_jobs = [os.path.splitext(x)[0] for x in os.listdir('Data')]
data_jobs_set = set(data_jobs)
i_possess_labels = clids.intersection(data_jobs)
i_miss = clids.difference(data_jobs)
label_miss = data_jobs_set.difference(clids)
print("Labeled Jobs I have: {}".format(len(i_possess_labels)))
print("Labeled Jobs I am Missing: {}".format(len(i_miss)))
print("Missing Labels for {} jobs".format(len(label_miss)))

## Lasso-Logit

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
labor_discrim_whole = os.path.join(os.getcwd(), 'Relevant_Files', 'Regular_Use', 'labor_discrim_whole.csv')
bag_of_words_labels = pd.read_csv(labor_discrim_whole, index_col = 'clid')

In [12]:
X_data = bag_of_words_labels.drop(['callback_non_minority', 'callback_minority', 'discriminated'], axis = 1).as_matrix()
y = bag_of_words_labels['discriminated'].as_matrix()

In [13]:
logit_clf = LogisticRegression(penalty='l1')
logit_clf.fit(X_data, y) #Fit on data
coef_values = logit_clf.coef_[logit_clf.coef_ != 0] #Obtain all non-removed coefficients

In [14]:
print(coef_values.shape)
print(logit_clf.coef_.shape)

(470,)
(1, 740)


In [15]:
X_features = bag_of_words_labels.drop(['callback_non_minority', 'callback_minority', 'discriminated'], axis = 1).columns.values
word_coef = np.column_stack((X_features,logit_clf.coef_.reshape((740))) )
word_coef_nonzero = word_coef[word_coef[:,1] != 0]
word_coef_df = pd.DataFrame(word_coef).to_csv('word_weights_all.csv')
word_coef_nonzero_df = pd.DataFrame(word_coef_nonzero).to_csv('word_weights_nonzero.csv')

## Lasso-Probit

In [2]:
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit

/home/jason/anaconda3/envs/labordiscrim/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#Example: http://www.statsmodels.org/stable/examples/notebooks/generated/discrete_choice_overview.html
#Other Examples: http://www.statsmodels.org/stable/examples/index.html#discrete

In [7]:
labor_discrim_whole = os.path.join(os.getcwd(), 'Relevant_Files', 'Regular_Use', 'labor_discrim_whole.csv')
bag_of_words_labels = pd.read_csv(labor_discrim_whole, index_col = 'clid')
Bag_of_Words = bag_of_words_labels.drop(['callback_non_minority', 'callback_minority', 'discriminated'], axis = 1).as_matrix()
X = sm.add_constant(Bag_of_Words, prepend = False)
y = bag_of_words_labels['discriminated'].as_matrix()

In [8]:
X

array([[ 0.,  3.,  1., ...,  0.,  0.,  1.],
       [ 1.,  1.,  0., ...,  0.,  0.,  1.],
       [ 0.,  3.,  1., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1., ...,  1.,  0.,  1.],
       [ 1.,  1.,  1., ...,  0.,  0.,  1.],
       [ 1.,  3.,  1., ...,  0.,  0.,  1.]])

In [5]:
probit_mod = sm.Probit(y, X)

In [6]:
probit_res = probit_mod.fit_regularized(method = 'l1', disp=True)

Iteration limit exceeded    (Exit mode 9)
            Current function value: 0.01132542159751891
            Iterations: 1001
            Function evaluations: 1008
            Gradient evaluations: 1001


LinAlgError: Singular matrix

In [ ]:
probit_res.summary()

In [ ]:
probit_mod2 = sm.Probit(endog, exog_2)

In [ ]:
probit_mod2.fit_regularized(method='l1',disp = 1, alpha=1) 